In [ ]:
import fiber
import pandas as pd
import numpy as np
import sys
import os
import pickle
import seaborn as sns

import shap

from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler 
from sklearn.model_selection import cross_val_predict, cross_validate, KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, precision_score, recall_score, accuracy_score, brier_score_loss, precision_recall_curve
from sklearn.compose import ColumnTransformer
from category_encoders import OneHotEncoder, TargetEncoder
from category_encoders import *
import category_encoders as ce

from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from sklearn.experimental import enable_iterative_imputer 

import sklearn.impute
from sklearn import impute
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
import inspect
import json
from pydoc import locate
import requests

import warnings
warnings.filterwarnings('ignore')

##set random seeds
np.random.seed(7)

pd.set_option('display.max_columns', 100)

In [ ]:
input_filename = 'Renal_ML_pipeline_final.pkl'
output_filename = 'results_with_renal_data_0_2_P.csv'
NA_removal_threshold = 80 ##atleast this many columns should have a non null value: 80 for unsupervised
prediction_window = 0
test_df_control_ratio = 2
train_df_control_ratio = 2
THRESHOLD = 0.2

xgb_param = {'silent': 0, 'n_jobs': -1, 'max_depth': 15, 'reg_alpha': 1, 'reg_lambda': 1, 'random_state': 42, 'learning_rate': 0.05, 'max_bin': 32,
    'colsample_bytree': 0.20, 'min_child_weight': 0.5, 'min_split_loss': 0.5, 'subsample': 0.5 }

catboost_param = {'max_depth': 5, 'random_state': 42, 'learning_rate': 0.01, 'max_bin': 32,
                  'objective': 'Logloss','eval_metric': 'AUC', 'iterations': 1200, 'verbose': 50, 
                  'l2_leaf_reg': 2, 'boosting_type': 'Plain', 'boost_from_average': False,
                  'grow_policy': 'Lossguide',
                  'min_data_in_leaf': 15,'max_leaves': 45,
                  'custom_metric': ['Logloss', 'AUC'] 
                 }

lgb_param_calibrated = {'objective': 'binary', # for binary classification
        'boost_from_average': False,
        'is_unbalance': True,
        'boosting': 'gbdt', # traditional gradient boosting decision tree
        'learning_rate': 0.05,
        'num_leaves': 250,
        'device': 'cpu', # you can use GPU to achieve faster learning
        'max_depth': 15, # <0 means no limit
        'max_bin': 32, # Small number of bins may reduce training accuracy but can deal with over-fitting
        'lambda_l1': 2, # L1 regularization
        'lambda_l2': 2, # L2 regularization
        'subsample_for_bin': 200, # number of samples for constructing bins
        'subsample': 1, # subsample ratio of the training instance
        'colsample_bytree': 0.2, # subsample ratio of columns when constructing the tree
        'min_split_gain': 0.5, # minimum loss reduction required to make further partition on a leaf node of the tree
        'min_child_weight': 1, # minimum sum of instance weight (hessian) needed in a leaf
        'min_child_samples': 5, # minimum number of data needed in a leaf
        'feature_fraction': 0.5,
        'metric' : 'auc',
        'reg_alpha': 1,
        'reg_lambda': 1,
        'bagging_fraction':0.8,
        'bagging_freq':10          
        }

lgb_param = {'num_leaves':40, 'objective':'binary','max_depth':7,'learning_rate':0.01,'max_bin':128, 'metric': ['auc', 'binary_logloss'],
 'n_jobs':-1, 'reg_alpha': 0.5, 'reg_lambda': 1, 'random_state': 42,'feature_fraction': 0.25}


def train_test_split(df, test_df_control, train_df_control):
    
    train_df = df[df.train_test == 'train']
    test_df = df[df.train_test == 'test']

    if test_df_control_ratio is not None:
        test_df_ht = test_df[test_df.Complication == '1']
        test_df_not_ht = test_df[test_df.Complication == '0']
        test_df_not_ht = test_df_not_ht.sample(int(test_df_control_ratio * test_df_ht.shape[0]), random_state=42)
        test_df = pd.concat([test_df_ht,test_df_not_ht])
        
    if train_df_control_ratio is not None:
        train_df_ht = train_df[train_df.Complication == '1']
        train_df_not_ht = train_df[train_df.Complication == '0']
        train_df_not_ht = train_df_not_ht.sample(int(train_df_control_ratio * train_df_ht.shape[0]), random_state=42)
        train_df = pd.concat([train_df_ht,train_df_not_ht])
    
    mrn_train = set(train_df['medical_record_number'])
    mrn_test = set(test_df['medical_record_number'])
    train_test_common = mrn_train.intersection(mrn_test)
    train_df = train_df.drop(columns=['train_test', 'medical_record_number'], errors = 'ignore')
    test_df = test_df.drop(columns=['train_test', 'medical_record_number'], errors = 'ignore')
    return train_df, test_df


class FriendlyNamesConverter:
    def rename_columns(self, df):
        replacements = {}
        for column in df.columns:
            replacements[column] = self.get(column)
        return replacements

    def get(self, feature):
        # does not support time window information inside feature name yet
        if feature.startswith(('age', 'gender', 'religion', 'race')):
            return feature.replace('_', ' ').replace('.', '|')

        split_name = feature.split('__')
        if len(split_name) > 1: 
            if split_name[1] in [
                i[0]
                for i in inspect.getmembers(
                    sys.modules['fiber.condition'],
                    inspect.isclass
                )
            ]:
                aggregation = split_name[0]
                split_name = split_name[1:]
            else:
                aggregation = None

            if len(split_name) == 3:
                class_name, context, code = split_name
                condition_class = locate(f'fiber.condition.{class_name}')
                description = self.get_description(condition_class, code, context)
                if  "Lipid panel" in description:
                    description = "Lipid panel"
            else:
                class_name, description = split_name

            if aggregation is not None: 
                return f'{class_name} | {description.capitalize()} ({aggregation})'
            else:
                return f'{class_name} | {description.capitalize()}'
        else:
            return feature

    def get_description(self, condition_class, code, context):
        return condition_class(
            code=code,
            context=context
        ).patients_per(
            condition_class.description_column
        )[
            condition_class.description_column.name.lower()
        ].iloc[0]


def get_column_names_from_ColumnTransformer(column_transformer):    
    col_name = []
    for transformer_in_columns in column_transformer.transformers_:#the last transformer is ColumnTransformer's 'remainder'
        raw_col_name = transformer_in_columns[2]
        if isinstance(transformer_in_columns[1],Pipeline): 
            transformer = transformer_in_columns[1].steps[-1][1]
        else:
            transformer = transformer_in_columns[1]
        try:
            names = transformer.get_feature_names()
        except AttributeError: # if no 'get_feature_names' function, use raw column name
            names = raw_col_name
        if isinstance(names,np.ndarray): # eg.
            col_name += names.tolist()
        elif isinstance(names,list):
            col_name += names    
        elif isinstance(names,str):
            col_name.append(names)
    return col_name


def get_shap_importanceplot(train_df, test_df):
    

    categorical_cols = [c for c in train_df.columns if train_df[c].dtype in [np.object] and c not in ['Complication']]
    numerical_cols = [c for c in train_df.columns if train_df[c].dtype in [np.float, np.int, 'uint8'] and c not in ['Complication']]
    train_df['Complication'] = pd.to_numeric(train_df['Complication'])
    test_df['Complication'] = pd.to_numeric(test_df['Complication'])
    
    print("length of column names are ::" + str(len(categorical_cols) + len(numerical_cols)))

    column_transformer = ColumnTransformer([
        ('num', StandardScaler(), numerical_cols),
        ('cat', TargetEncoder(), categorical_cols),])
        
    
    xgb_classifier = XGBClassifier(**xgb_param)

    retro_train = train_df[train_df.columns.difference(['Complication'])]
    retro_label = train_df['Complication']
    pros_train = test_df[test_df.columns.difference(['Complication'])]
    pros_label = test_df['Complication']

    
    preprocessed_data = column_transformer.fit_transform(retro_train, retro_label)
    column_names = get_column_names_from_ColumnTransformer(column_transformer)

    preprocessed_data = pd.DataFrame(preprocessed_data, columns = column_names)

    xgb_classifier.fit(preprocessed_data, train_df['Complication'])
    
    shap.initjs()
    
    shap_values = shap.TreeExplainer(xgb_classifier).shap_values(preprocessed_data)
    f = plt.figure()
    shap.summary_plot(shap_values, preprocessed_data, plot_type = 'dot')
    
    f.savefig((f'shap_final_xgb_extraBP_TargetEnc_{input_filename}.png'), bbox_inches='tight', dpi=600)
    
    #save SHAP values as dataframe
    shap_sum = np.abs(shap_values).mean(axis=0)
    importance_df = pd.DataFrame([preprocessed_data.columns.tolist(), shap_sum.tolist()]).T
    importance_df.columns = ['column_name', 'shap_importance']
    importance_df = importance_df.sort_values('shap_importance', ascending=False)
    print (importance_df.head())
    importance_df.to_csv('/home/kiwitn01/master_thesis_hypertension-complications/Machine_Learning/ML_Pipeline/SHAP_agg_data/Final_SHAP/Renal_final_extraBP_TargetEnc.csv')


if __name__ == "__main__":

    #df =  pd.read_pickle(os.path.join('/home/kiwitn01/master_thesis_hypertension-complications/Case_Control_Cohort_Creation/For_ML_Pipeline/Split_2012', input_filename))
    df =  pd.read_pickle(os.path.join('/home/kiwitn01/master_thesis_hypertension-complications/Case_Control_Cohort_Creation/For_ML_Pipeline/Split_2012/with_extra_BP_and_demographic_data', input_filename))
           
    print(df.shape)
    df.to_pickle(os.path.join('/home/kiwitn01/master_thesis_hypertension-complications/Machine_Learning/plots', output_filename))
    
    df = df.dropna(axis=0, thresh = NA_removal_threshold)
    df = df.drop(['marital_status_code'], axis = 1)
    
    #rename_dict = FriendlyNamesConverter().rename_columns(df)
    #df = df.rename(columns=rename_dict)
    #df = df.loc[:,~df.columns.duplicated()]
    
    print(df.shape)
    print("final dataframe shape after dropping NAs" + str(df.shape))
    print(pd.crosstab(df.train_test, df.Complication))

    train_df, test_df = train_test_split(df, test_df_control_ratio, train_df_control_ratio)
    
    get_shap_importanceplot(train_df,test_df)
    